<a href="https://colab.research.google.com/github/katieprice14/computer-vision-system/blob/master/modifing_deeplearning_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports and mounting to drive

In [ ]:
from google.colab import drive
import os
import openpyxl
import numpy as np

drive.mount('/content/drive')

## Extracting data and Track

This opens the input data in a binary file (directory), and the output file (excel file)

Opens a binary and csv file to write data and track

Opens the excel file and extracts the width, length, height, volume and adds it to an array as data type float 32

In [ ]:
directory = r"/content/drive/MyDrive/VT Animal Science: Computer vision system for beef cattle/Deep Learning Scripts and Resources/data/bin"
directory_output = r"/content/drive/MyDrive/VT Animal Science: Computer vision system for beef cattle/Deep Learning Scripts and Resources/data/type_cow"

allcowsb = open(directory + "/allcows.bin", "wb")
allcowsc = open(directory + "/allcows.csv", "w")

csv_files = [x for x in os.listdir(directory) if ".csv" in x]
start = 0

wb = openpyxl.load_workbook(directory_output + '/' + id + ".xlsx")
sheet = wb.active
for idx,csv in enumerate(csv_files):
  if idx == 2:
    break
  track = open(directory + "/" + csv,"r")
  print(directory + "/" + csv)
  id = csv[:-10]
  bin = open(directory + "/" + id + ".bin", "rb")


  width = sheet['F2'].value
  length = sheet['G2'].value
  height = sheet['I2'].value
  volume = sheet['J2'].value
  array = np.array([width,length,height,volume],dtype=np.float32)

## Reading Bin and CSV files

Reads the values in the Bin file (actual measurements) and the csv file (tracking file)

writes the information from the bin file and the csv file into a bin and csv file that compiles all of the cows

Read_bytes tracks the number of bytes used in each image from the csv file

wroteI - writes the length of file in bytes into allcowsb

wroteO - Writes the output data in bytes into the binary file

In the CSV File, writes the start point, length of file, start point and output data

In [ ]:
  for row in track:
    cell = row.split(",")
    bin.seek(int(cell[0]))
    read_bytes = bin.read(int(cell[1]))
    data = np.frombuffer(read_bytes)
    data = data.astype(np.float32)
    wroteI = allcowsb.write(data.tobytes())
    wroteO = allcowsb.write(array.tobytes())    
    allcowsc.write(str(start)+","+str(wroteI)+",")
    start += wroteI 
    allcowsc.write(str(start)+","+str(wroteO)+"\n")
    start += wroteO
  track.close()
  bin.close()
allcowsb.close()
allcowsc.close()

## Adding to the Complete Binary File and shuffling

Opens the binary file and shuffles each line
Writes the shuffled lines into a new complete file

In [ ]:
with open(directory + "/allcows.bin", "rb") as allcowsbin:
  with open(directory + "/allcows.csv", "r") as allcowscsv:
    with open(directory + "/complete.bin", "wb") as complete:
      csv_array = allcowscsv.readlines()
      shuffled = np.random.shuffle(csv_array)
      for line in csv_array:
        cell = line.strip().split(',')
        allcowsbin.seek(int(cell[0]))
        one = allcowsbin.read(int(cell[1]))
        allcowsbin.seek(int(cell[2]))
        two = allcowsbin.read(int(cell[3]))
        complete.write(one)
        complete.write(two)

Combines the input data in into a singular batch

Combines the output data into a singular batch

Writes the start bytes, length, and end bytes into a CSV file

In [ ]:
batch_size = 32
start_bytes = 0
input_array = np.zeros((batch_size,224,224,3)) #Change from 3 to 1
output_array = np.zeros((batch_size,4))
with open(directory + "/complete.bin", "rb") as complete:
  with open(directory + "/allcows.csv", "r") as allcowscsv:
    with open(directory + "/allcows.bin", "wb") as allcowsbin:
      with open(directory + "/cows.csv","w") as cows:
        csv_array = allcowscsv.readlines()
        for idx,line in enumerate(csv_array):
          index = idx % batch_size
          cell = line.strip().split(',')
          complete.seek(int(cell[0]))
          one = complete.read(int(cell[1]))
          one = np.frombuffer(one,dtype = np.float32)
          one = one.reshape((224,224,3))
          complete.seek(int(cell[2]))
          two = complete.read(int(cell[3]))
          two = np.frombuffer(two,dtype = np.float32)
          two = two.reshape((4))   
          input_array[index,:,:,:] = one
          output_array[index,:] = two  
          if index == batch_size-1:
            write_input = allcowsbin.write(input_array.tobytes())
            write_output = allcowsbin.write(output_array.tobytes())
            cows.write(str(start_bytes))
            cows.write("," + str(write_input))
            start_bytes += write_input
            cows.write("," + str(start_bytes))
            cows.write("," + str(write_output))
            start_bytes += write_output
        index += 1            